## Classification of active municipalities

In [24]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import matplotlib.pyplot as plt
import ydata_profiling
from ydata_profiling import ProfileReport
import seaborn as sns
import numpy as np

In [15]:
df = pd.read_csv("../data/interim/twitter_weather_electrical.csv")
df['geometry'] = df['geometry'].apply(wkt.loads)
df = gpd.GeoDataFrame(df).set_crs('EPSG:4326')
df['hour_blocks'] = df['time'].str.split(':').str[0].astype(int)

In [16]:
df = df.drop(['user', 'language', 'geometry_y'], axis=1)

Calculate hour categories

In [17]:
def categorize_hour(hour):
    if 2 <= hour < 8:
        return 'A'  # 2-7
    elif 8 <= hour < 14:
        return 'B'  # 8-13
    elif 14 <= hour < 20:
        return 'C'  # 14-19
    elif (20 <= hour <= 23) or (0 <= hour <= 1):  # 20-23 or 00-01
        return 'D'
    else:
        return None  # Fallback in case something doesn't match

# Apply the function to create the hour_category
df['hour_category'] = df['hour_blocks'].apply(categorize_hour)

In [18]:
df

,date,time,timestamp_x,municipality.name,hour_blocks,geometry,station,elevation,timestamp_y,minTemperature,...,precipitation,minWind,maxWind,temperature,wind,cellId,tot_curr_cell,NR_SITES,curr_site,hour_category
0,2013-11-01,00:04,1383260656,Trento,0,POINT (11.13000 46.07000),T0129,312,1383260400,11.7,...,0.0,0.2,2.0,13.2,1@47,5201,90.971940,1288.0,0.070630,D
1,2013-11-01,00:01,1383260474,Pinzolo,0,POINT (10.83000 46.23000),T0071,905,1383260400,4.5,...,0.0,0.0,2.5,8.4,0.1@205,7167,1.784586,24.0,0.074358,D
2,2013-11-01,00:43,1383262983,Cavalese,0,POINT (11.46000 46.29000),T0367,958,1383260400,7.0,...,0.0,0.0,1.8,8.8,NaN,8151,0.000000,0.0,0.000000,D
3,2013-11-01,00:38,1383262717,Rovereto,0,POINT (11.04000 45.89000),T0147,203,1383260400,11.0,...,0.0,0.2,1.9,13.2,0.6@7,2855,47.591226,995.0,0.047830,D
4,2013-11-01,00:45,1383263140,San Michele all'Adige,0,POINT (11.12000 46.20000),T0408,204,1383260400,9.6,...,0.0,NaN,NaN,11.6,NaN,6838,0.100372,4.0,0.025093,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27931,2013-12-31,00:25,1388445948,Vigo di Fassa,0,POINT (11.68000 46.42000),T0096,1205,1388444400,-10.0,...,0.0,NaN,NaN,-7.8,NaN,9922,0.000000,0.0,0.000000,D
27932,2013-12-29,13:12,1388319166,Predazzo,13,POINT (11.60000 46.31000),T0389,1000,1388271600,0.4,...,0.0,NaN,NaN,3.4,NaN,8513,0.000000,0.0,0.000000,B
27933,2013-11-19,00:51,1384818699,Sant'Orsola Terme,0,POINT (11.30000 46.11000),T0139,925,1384815600,4.3,...,0.0,NaN,NaN,4.8,NaN,5799,0.000000,0.0,0.000000,D
27934,2013-11-23,09:49,1385196576,Sant'Orsola Terme,9,POINT (11.30000 46.11000),T0139,925,1385161200,-1.6,...,0.0,NaN,NaN,0.8,NaN,5799,0.000000,0.0,0.000000,B


In [19]:
df_grouped = df.groupby(['date', 'municipality.name', 'hour_category']).agg({
    'temperature':'mean',
    'minTemperature':'mean',
    'maxTemperature':'mean',
    'precipitation':'mean',
    'timestamp_x': 'count' 
}).rename(columns={'timestamp_x': 'tweet_count'}).reset_index()

In [20]:
df_grouped

,date,municipality.name,hour_category,temperature,minTemperature,maxTemperature,precipitation,tweet_count
0,2013-11-01,Andalo,A,9.200000,8.5,12.000000,0.0,1
1,2013-11-01,Baselga di Pinè,B,11.533333,8.0,13.733333,0.0,3
2,2013-11-01,Baselga di Pinè,C,11.433333,7.3,12.966667,0.0,3
3,2013-11-01,Baselga di Pinè,D,9.350000,6.6,12.200000,0.0,2
4,2013-11-01,Bedollo,B,14.120000,8.7,14.500000,0.0,5
...,...,...,...,...,...,...,...,...
5026,2013-12-31,Vigo di Fassa,B,-7.450000,-10.0,0.300000,0.0,4
5027,2013-12-31,Vigo di Fassa,C,-3.966667,-10.0,0.300000,0.0,3
5028,2013-12-31,Vigo di Fassa,D,-8.125000,-10.0,0.300000,0.0,4
5029,2013-12-31,Villa Agnedo,C,0.533333,-3.3,4.900000,0.0,3


In [27]:
conditions = [
    (df_grouped['tweet_count'] >= df_grouped['tweet_count'].quantile(0.9)),  # Top 10%
    (df_grouped['tweet_count'] < df_grouped['tweet_count'].quantile(0.9)) & 
    (df_grouped['tweet_count'] >= df_grouped['tweet_count'].quantile(0.5)),  # Middle 40%
    (df_grouped['tweet_count'] < df_grouped['tweet_count'].quantile(0.5))    # Bottom 50%
]

In [29]:
choices = ['High', 'Medium', 'Low']
df_grouped['class'] = np.select(conditions, choices, default='Low')

In [30]:
df_grouped

,date,municipality.name,hour_category,temperature,minTemperature,maxTemperature,precipitation,tweet_count,class
0,2013-11-01,Andalo,A,9.200000,8.5,12.000000,0.0,1,Low
1,2013-11-01,Baselga di Pinè,B,11.533333,8.0,13.733333,0.0,3,Medium
2,2013-11-01,Baselga di Pinè,C,11.433333,7.3,12.966667,0.0,3,Medium
3,2013-11-01,Baselga di Pinè,D,9.350000,6.6,12.200000,0.0,2,Medium
4,2013-11-01,Bedollo,B,14.120000,8.7,14.500000,0.0,5,Medium
...,...,...,...,...,...,...,...,...,...
5026,2013-12-31,Vigo di Fassa,B,-7.450000,-10.0,0.300000,0.0,4,Medium
5027,2013-12-31,Vigo di Fassa,C,-3.966667,-10.0,0.300000,0.0,3,Medium
5028,2013-12-31,Vigo di Fassa,D,-8.125000,-10.0,0.300000,0.0,4,Medium
5029,2013-12-31,Villa Agnedo,C,0.533333,-3.3,4.900000,0.0,3,Medium


In [35]:
#df_grouped[(df_grouped['municipality.name'] == 'Trento') & (df_grouped['class'] == 'High')]

Remove last day :C

In [37]:
day_to_predict = '2013-12-31'

In [41]:
data = df_grouped[df_grouped['date'] != day_to_predict]

In [43]:
data

,date,municipality.name,hour_category,temperature,minTemperature,maxTemperature,precipitation,tweet_count,class
0,2013-11-01,Andalo,A,9.200000,8.5,12.000000,0.0,1,Low
1,2013-11-01,Baselga di Pinè,B,11.533333,8.0,13.733333,0.0,3,Medium
2,2013-11-01,Baselga di Pinè,C,11.433333,7.3,12.966667,0.0,3,Medium
3,2013-11-01,Baselga di Pinè,D,9.350000,6.6,12.200000,0.0,2,Medium
4,2013-11-01,Bedollo,B,14.120000,8.7,14.500000,0.0,5,Medium
...,...,...,...,...,...,...,...,...,...
4867,2013-12-30,Volano,B,3.600000,0.5,8.100000,0.0,1,Low
4868,2013-12-30,Volano,C,3.500000,0.5,8.100000,0.0,1,Low
4869,2013-12-30,Zambana,D,0.400000,0.1,8.800000,0.0,1,Low
4870,2013-12-30,Ziano di Fiemme,C,-1.100000,-3.7,2.400000,0.0,1,Low


Divide between features and target

In [47]:
X = data[['date', 
          'municipality.name', 
          'hour_category', 
          'temperature', 
          'minTemperature', 
          'maxTemperature', 
          'precipitation', 
          'tweet_count' 
]]
Y = data['class']  # Target is the multi-class label (High, Medium, Low)

In [48]:
Y

0          Low
1       Medium
2       Medium
3       Medium
4       Medium
         ...  
4867       Low
4868       Low
4869       Low
4870       Low
4871    Medium
Name: class, Length: 4872, dtype: object

## Random forest let's go

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

In [57]:
# Separate categorical features
categorical_features = ['date', 'municipality.name', 'hour_category']  # Any string or categorical features
numerical_features = ['temperature', 'minTemperature', 'maxTemperature', 'precipitation', 'tweet_count']

In [58]:
# Scale the numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(X[numerical_features])

In [59]:
# Convert the scaled numerical features back to a DataFrame
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_features)

In [64]:
# Combine the scaled numerical features and encoded categorical features
X = pd.concat([scaled_numerical_df, X[categorical_features]], axis=1)

In [66]:
X
# it is possible to separate month and day from date

,temperature,minTemperature,maxTemperature,precipitation,tweet_count,date,municipality.name,hour_category
0,1.359660,2.083997,1.183481,-0.236583,-0.403825,2013-11-01,Andalo,A
1,1.910475,1.960059,1.636554,-0.236583,-0.225801,2013-11-01,Baselga di Pinè,B
2,1.886869,1.786544,1.436156,-0.236583,-0.225801,2013-11-01,Baselga di Pinè,C
3,1.395069,1.613030,1.235759,-0.236583,-0.314813,2013-11-01,Baselga di Pinè,D
4,2.521093,2.133573,1.836952,-0.236583,-0.047776,2013-11-01,Bedollo,B
...,...,...,...,...,...,...,...,...
4867,0.037703,0.100976,0.164067,-0.236583,-0.403825,2013-12-30,Volano,B
4868,0.014097,0.100976,0.164067,-0.236583,-0.403825,2013-12-30,Volano,C
4869,-0.717701,0.001825,0.347039,-0.236583,-0.403825,2013-12-30,Zambana,D
4870,-1.071797,-0.940111,-1.325846,-0.236583,-0.403825,2013-12-30,Ziano di Fiemme,C
